In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

## 데이터 수정

In [4]:
for df in [df_train,df_test]:
    # df['date'] = df['date'].apply(lambda x: x[0:8])
    # 일짜까지 되있는걸 월까지
    df['date'] = df['date'].apply(lambda x: x[0:6])
    df['data_y'] = ''
    df['data_m'] = ''
    df['data_y'] = df['date'].apply(lambda x : x[0:4])
    df['data_m'] = df['date'].apply(lambda x : x[4:6])
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: np.nan if x == 0 else x)
    df['yr_renovated'] = df['yr_renovated'].fillna(df['yr_built'])
for df in [df_train]:
    # 1000단위 대로 반올림
    df['price'] = df['price'].apply(lambda x: round(x, -3))

In [5]:
for df in [df_train,df_test]:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')

In [6]:
df_train['per_price'] = df_train['price']/df_train['sqft_total_size']
zipcode_price = df_train.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()
df_train = pd.merge(df_train,zipcode_price,how='left',on='zipcode')
df_test = pd.merge(df_test,zipcode_price,how='left',on='zipcode')

for df in [df_train,df_test]:
    df['mean'] = df['mean'] * df['sqft_total_size']
    df['var'] = df['var'] * df['sqft_total_size']

### 라벨 인코더

In [7]:
from sklearn.preprocessing import LabelEncoder
# LE = LabelEncoder()
# LE.fit(df_train[['date']])
# df_train[['date']] = LE.transform(df_train[['date']])
# LE.fit(df_train[['zipcode']])
# df_train[['zipcode']] = LE.transform(df_train[['zipcode']])

### 상호작용변수 추가

In [8]:
def geogege(data):
    data['zipcode'] = data['zipcode'].astype(str)  
    data['zipcode-3'] = data['zipcode'].apply(lambda x : str(x[2:3])).astype(int)
    data['zipcode-4'] = data['zipcode'].apply(lambda x : str(x[3:4])).astype(int)
    data['zipcode-5'] = data['zipcode'].apply(lambda x : str(x[4:5])).astype(int)
    data['zipcode-34'] = data['zipcode'].apply(lambda x : str(x[2:4])).astype(int)
    data['zipcode-45'] = data['zipcode'].apply(lambda x : str(x[3:5])).astype(int)
    data['zipcode-35'] = data['zipcode'].apply(lambda x : str(x[2:5])).astype(int)
#     data.drop('zipcode', axis=1, inplace=True)
    return data

In [9]:
def ppcca1(data):
    pca2 = PCA(n_components=2)
    coord_test = data[['lat','long']]
    
    principalComponents_updated_test = pca2.transform(coord_test)
    data['coord_pca1']= ''
    data['coord_pca2']= ''
    data['coord_pca1']= principalComponents_updated_test[:, 0]
    data['coord_pca2']= principalComponents_updated_test[:, 1]
    return data

In [10]:
def ppcca2(data):
    pca1 = PCA(n_components=2)

    principalComponents_updated_test = pca1.transform(data)
    data['pca1']= ''
    data['pca2']= ''
    data['pca1']= principalComponents_updated_test[:, 0]
    data['pca2']= principalComponents_updated_test[:, 1]
    return data

In [11]:
from sklearn.decomposition import PCA

In [12]:
df_train_length = len(df_train)
concat_dataset = pd.concat(objs=[df_train, df_test], axis=0)

# ZIP divide
concat_dataset = geogege(concat_dataset)

# LabelEncoder
LE = LabelEncoder()
# LE.fit(df_train[['date']])
# concat_dataset[['date']] = LE.transform(concat_dataset[['date']])
# LE.fit(df_train[['zipcode']])
# concat_dataset[['zipcode']] = LE.transform(concat_dataset[['zipcode']])
concat_dataset['date_zipcode'] = concat_dataset['date'].map(str) + "_" + concat_dataset['zipcode'].map(str)
LE.fit(concat_dataset[['date_zipcode']])
concat_dataset[['date_zipcode']] = LE.transform(concat_dataset[['date_zipcode']])

# PCA
# concat_dataset = ppcca1(concat_dataset)
# concat_dataset = ppcca2(concat_dataset)


# concat_dataset[concat_dataset.columns.difference(['per_price','price'])]

# df_train = concat_dataset[:df_train_length]
# df_test = concat_dataset[df_train_length:]

In [20]:
pca = PCA(n_components=6)
pca1 = pca.fit(concat_dataset[concat_dataset.columns.difference(['per_price','price'])])
df3 = pca.transform(pca1)

ValueError: Expected 2D array, got scalar array instead:
array=PCA(copy=True, iterated_power='auto', n_components=6, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False).
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [23]:
pca1.transform()

TypeError: transform() missing 1 required positional argument: 'X'

In [ ]:
pca2 = PCA(n_components=2)
coord_test = concat_dataset[['lat','long']]

principalComponents_updated_test = pca2.transform(coord_test)
df3['coord_pca1']= ''
df3['coord_pca2']= ''
df3['coord_pca1']= principalComponents_updated_test[:, 0]
df3['coord_pca2']= principalComponents_updated_test[:, 1]

In [14]:


X_train = df3[:df_train_length]
y_train
df_test = df3[df_train_length:]

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

### 월별 + ZIPCODE를 활용한 더미변수 추가하여 테스트
위도 + 경도로 위치값을 대신하고 싶었지만 어떻게 구역을 나눠야 할지 모르기때문에 ZIPCODE로 대신함

## XGboost

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import xgboost as xgb

# X_train = [c for c in df_train.columns if c not in ['id','price','per_price']]
# y_train = df_train['price']

In [ ]:
model= xgb.XGBRegressor(
    n_estimators=50000,
    num_round_boost=500,
    verbosity=0,
    reg_lambda=10,
    reg_alpha=0.01,
    learning_rate=0.005,
    seed=2019,
    colsample_bytree=0.8,
    colsample_bylevel=0.8,
    subsample=0.9,
    n_jobs=-1,
    gamma=0.005,
    nthread=-1
)

model.fit(X_train,y_train, verbose=False, eval_set=watchlist,
              eval_metric='rmse',
              verbose=2000,
              early_stopping_rounds=1000)

In [ ]:
score=mse(np.exp(model.predict(X_test)),np.exp(y_test))**0.5
pred=np.exp(model.predict(Y_test))

print("RMSE unseen : {}".format(xgb_score))